<img width="800px" src="../fidle/img/header.svg"></img>

# <!-- TITLE --> [K3IMDB4] - Reload embedded vectors
<!-- DESC --> Retrieving embedded vectors from our trained model, using Keras 3 and PyTorch
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - The objective is to retrieve and visualize our embedded vectors
 - For this, we will use our **previously saved model**.

## What we're going to do :

 - Retrieve our saved model
 - Extract vectors and play with


## Step 1 - Init python stuff

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'torch'

import keras

import json,re
import numpy as np

import fidle

# Init Fidle environment
run_id, run_dir, datasets_dir = fidle.init('K3IMDB4')

### 1.2 - Parameters
The words in the vocabulary are classified from the most frequent to the rarest.  
`vocab_size` is the number of words we will remember in our vocabulary (the other words will be considered as unknown).  
`review_len` is the review length  
`saved_models` where our models were previously saved  
`dictionaries_dir` is where we will go to save our dictionaries. (./data is a good choice)

In [ ]:
vocab_size           = 5000
review_len           = 256

saved_models         = './run/K3IMDB2'
dictionaries_dir     = './data'

Override parameters (batch mode) - Just forget this cell

In [ ]:
fidle.override('vocab_size', 'review_len', 'saved_models', 'dictionaries_dir')

## Step 2 - Get the embedding vectors !

### 2.1 - Load model and dictionaries
Note : This dictionary is generated by [02-Embedding-Keras](02-Keras-embedding.ipynb) notebook.

In [ ]:
model = keras.models.load_model(f'{saved_models}/models/best_model.keras')
print('Model loaded.')

with open(f'{dictionaries_dir}/word_index.json', 'r') as fp:
    word_index = json.load(fp)
    index_word = { i:w      for w,i in word_index.items() }
    print('Dictionaries loaded. ', len(word_index), 'entries' )

### 2.2 - Retrieve embeddings

In [ ]:
embeddings = model.layers[0].get_weights()[0]
print('Shape of embeddings : ',embeddings.shape)

### 2.3 - Build a nice dictionary

In [ ]:
word_embedding = { index_word[i]:embeddings[i] for i in range(vocab_size) }

## Step 3 - Have a look !
#### Show embedding of a word :

In [ ]:
word_embedding['nice']

#### Few usefull functions to play with

In [ ]:
# Return a l2 distance between 2 words
#
def l2w(w1,w2):
    v1=word_embedding[w1]
    v2=word_embedding[w2]
    return np.linalg.norm(v2-v1)

# Show distance between 2 words 
#
def show_l2(w1,w2):
    print(f'\nL2 between [{w1}] and [{w2}] : ',l2w(w1,w2))

# Displays the 15 closest words to a given word
#
def neighbors(w1):
    v1=word_embedding[w1]
    dd={}
    for i in range(4, 1000):
        w2=index_word[i]
        dd[w2]=l2w(w1,w2)
    dd= {k: v for k, v in sorted(dd.items(), key=lambda item: item[1])}
    print(f'\nNeighbors of [{w1}] : ', list(dd.keys())[1:15])
    

#### Examples

In [ ]:
show_l2('nice', 'pleasant')
show_l2('nice', 'horrible')

neighbors('horrible')
neighbors('great')


In [ ]:
fidle.end()

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>